Pablo Jimeno - UPV/EHU 2016

In [ ]:
#%run 'Utilities_main.ipynb'

In [ ]:
### Load redMaPPer catalog:

redm_filedir = cat_dir + '/redMaPPer_DR12_v6.3.fits'
raw_redm = import_redMaPPer(redm_filedir)

print
print '*'*50

## Compute weighted spectroscopic redshift
raw_redm['w_spec_z'] = weighted_redshift(raw_redm)

## Reduce centers to most probable CG position
raw_redm['ra']  = raw_redm['ra'][:,0]
raw_redm['dec'] = raw_redm['dec'][:,0]


## Export catalogue
if 0:
    catalog_out = np.array((raw_redm['id'], 
                            raw_redm['ra'],
                            raw_redm['dec'],
                            raw_redm['w_spec_z'],
                            raw_redm['photo_z'],
                            raw_redm['photo_z_err'],
                            raw_redm['rich'],
                            raw_redm['rich_err'])).T

    # HEADER: ID / RA / DEC / W_SPEC_Z / PHOTO_Z / PHOTO_Z_ERR / RICHNESS / RICHNESS_ERR
    np.savetxt(cat_dir + '/redMaPPer_DR12_v6.3.txt', catalog_out,
               fmt='%.i\t%.7f\t%.7f\t%.6f\t%.6f\t%.6f\t%.2f\t  %.2f')

In [ ]:
### Define Planck maps:

planck_maps = {
     30  : 'LFI_SkyMap_030-field-IQU_1024_R2.01_full.fits',
     44  : 'LFI_SkyMap_044-field-IQU_1024_R2.01_full.fits',
     70  : 'LFI_SkyMap_070-field-IQU_2048_R2.01_full.fits',
    100  : 'HFI_SkyMap_100-field-IQU_2048_R2.02_full.fits',
    143  : 'HFI_SkyMap_143-field-IQU_2048_R2.02_full.fits',
    217  : 'HFI_SkyMap_217-field-IQU_2048_R2.02_full.fits',
    353  : 'HFI_SkyMap_353-field-IQU_2048_R2.02_full.fits',
    545  : 'HFI_SkyMap_545-field-Int_2048_R2.02_full.fits',
    857  : 'HFI_SkyMap_857-field-Int_2048_R2.02_full.fits',
}

## Other maps...
#'LFI_SkyMap_030-BPassCorrected-field-IQU_0256_R2.01_full.fits',
#'LFI_SkyMap_044-BPassCorrected-field-IQU_0256_R2.01_full.fits',
#'LFI_SkyMap_070-BPassCorrected-field-IQU_0256_R2.01_full.fits',
    
print '\nAvailable Planck maps, in planck_maps:\n'
print 'map_tag \t    map_file\n'
for map_tag, map_file in planck_maps.items():
    print '%s \t--> %s' % (map_tag, map_file)
print
print '*'*50
    
    
### Define patch properties:

img_pix = 256
zoom_factor = 50

In [ ]:
#-------------------------------------------------------------------
def generate_new_patches():
    """Refresh the patches folders."""

    for band in planck_maps.keys():
        patches_dir = '%s/Planck_patchs/%s' % (data_dir, band)
        if os.path.exists(patches_dir):
            shutil.rmtree(patches_dir)
        os.makedirs(patches_dir)
    
    
#-------------------------------------------------------------------
def generate_new_images():
    """Refresh the PNG folders.""" 

    for band in planck_maps.keys():
        png_dir = '%s/Planck/PNG/%s' % (img_dir, band)
        if os.path.exists(png_dir):
            shutil.rmtree(png_dir)
        os.makedirs(png_dir)
               

#-------------------------------------------------------------------
def generate_new_SZ_patches():
    """Refresh the SZ patches folder.""" 

    SZ_patches_dir = '%s/Planck_patchs/SZ' % data_dir
    if os.path.exists(SZ_patches_dir):
        shutil.rmtree(SZ_patches_dir)
    os.makedirs(SZ_patches_dir)
    
    
#-------------------------------------------------------------------
def generate_new_SZ_images():
    """Refresh the SZ PNG folder.""" 

    SZ_png_dir = '%s/Planck/PNG/SZ' % img_dir
    if os.path.exists(SZ_png_dir):
        shutil.rmtree(SZ_png_dir)
    os.makedirs(SZ_png_dir)
    
    
#-------------------------------------------------------------------
def define_tools():
    """Define some filtering & miscellaneous tools"""

    FOV = { # zoom_factor : degrees
        50 :  2.51136762, 
        20 :  6.28106129,
        10 : 12.58109431
    }

    FOV_deg = FOV[zoom_factor]
    FOV_arcmin = FOV_deg*60.0
    pix_arcmin = FOV_arcmin/img_pix

    Conv = { # band : K_CMB to MJy/sr
        545 : 57.2074,
        857 : 2.20780
    }

    FWHM = { # band : arcminutes
        30 : 33.0,
        44 : 24.0,
        70 : 14.0,
        100 : 9.66,
        143 : 7.27,
        217 : 5.01,
        353 : 4.86,
        545 : 4.84,
        857 : 4.63
    }

    FWHM_pix = {}
    for band, val in FWHM.items():
        FWHM_pix[band] = val/pix_arcmin

    FWHM_smooth     = 10.0 # arcmin
    FWHM_smooth_pix = FWHM_smooth/pix_arcmin

    for band in [100, 143, 217, 353, 545, 857]:
        FWHM_pix[band] = np.sqrt(FWHM_smooth_pix**2 - FWHM_pix[band]**2)

    FFTa = {}
    for band in [100, 143, 217, 353, 545, 857]:
        FFTa[band] = create_antenna(img_pix, FWHM_pix[band], display=False)[1]   

    MHW_scale = 10.0
    FFTMHW = create_MHW(img_pix, MHW_scale, display=False)[1]
    
    SZ_response = {
    30  : -5.323,
    44  : -5.1592,
    70  : -4.7076,
    100 : -4.0070,
    143 : -2.762,
    217 : 0.1886,
    353 : 6.124,
    545 : 14.77,
    857 : 29.62
    }
    
    return FFTa, FFTMHW, Conv, SZ_response


#-------------------------------------------------------------------

FFTa, FFTMHW, Conv, SZ_response = define_tools()

## Refresh data... CAUTION!!!
if 0: # Are you sure?
    if 0: # Really?
        generate_new_patches() # Redo patches
        generate_new_images()
        
## Refresh SZ data... CAUTION!!!
if 0: # Are you sure?
    if 0: # Really?
        generate_new_SZ_patches() # Redo SZ patches
        generate_new_SZ_images()

In [ ]:
#-------------------------------------------------------------------
def define_redm_subsample(rich_min, rich_max, z_min, z_max):
    """Make sample selection and define redm subsample"""

    ## Richness mask:
    rich_mask = (raw_redm['rich'] >= rich_min) & (raw_redm['rich'] < rich_max)
    
    ## Redshift mask:
    photo_z_mask = (raw_redm['photo_z'] >= z_min) & (raw_redm['photo_z'] < z_max)
    #spec_z_mask = (raw_redm['w_spec_z'] == -1.) | (raw_redm['w_spec_z'] >= z_min)
    redshift_z_mask = photo_z_mask# & spec_z_mask
    
    ## Global mask:
    global_mask = rich_mask & redshift_z_mask

    ## Construct sample:
    subsample = {}
    mask = global_mask

    for key in raw_redm.keys():
        subsample[key] = raw_redm[key][mask]
        
    N_clu = len(subsample['id'])
    rich_mean = np.mean(subsample['rich'])
    z_mean = np.mean(subsample['photo_z'])
            
    subsample_meta = [rich_min, rich_max, z_min, z_max, N_clu, rich_mean, z_mean]
    
    print '\nSAMPLE TAG: %s' % sub_tag(subsample_meta)
    print '\nConstructing redMaPPer subsample...\n'
    print 'R range: [%.1f, %.1f)' % (rich_min, rich_max)
    print 'z range: [%.3f, %.3f)' % (z_min, z_max)
    print '\nClusters inside R range: %i' % rich_mask.sum()
    print 'Clusters inside z range: %i' % redshift_z_mask.sum()
    print 'Clusters inside R range & z range: %i\n' % N_clu
    print '<R> = %.1f' % rich_mean
    print '<z> = %.3f\n' % z_mean
    print '-'*20
    
    return subsample['id'], subsample_meta

    
#-------------------------------------------------------------------        
def create_patches(redm_ids, band, produce_images=False):
    """If missing, obtain patches of clusters with redm_ids from Planck band map.""" 

    print '\nCreating patches for %s map...\n' % band
    
    planck_map, Nside = read_planck_map(planck_maps[band], display=False)

    print '\nProcessing clusters...\n'

    for i, clu_id in enumerate(redm_ids):

        print '\r%.2f completed' % (100.*i/len(redm_ids)),

        ## Obtain patches:
        patch_file = '%s/Planck_patchs/%s/Patch_%06d_%02d.npy' % (data_dir, band, clu_id, zoom_factor)
        if not os.path.isfile(patch_file):
            map2D = get_planck_patch(planck_map, clu_id, img_pix, zoom_factor)[0]
            np.save(patch_file, map2D)
                        
        ## Print image in img_dir:
        if produce_images:
            map2D = cluster_patch(clu_id, band, zoom_factor)
            png_file = '%s/Planck/PNG/%s/IMG_%06d_%02d.png' % (img_dir, band, clu_id, zoom_factor)
            fig = plot_image_data(map2D, lognorm=False)
            fig.savefig(png_file)
            plt.close(fig)
    
    print '\r100.00 completed\n'
    print '-'*20

    
#-------------------------------------------------------------------        
def smooth_map(map2D, band):
    """Smooth map to common resolution."""
    
    FFTm = fftpack.fft2(map2D)
    FFTm = FFTm * FFTa[band]
    map2D_smooth = fftpack.ifft2(FFTm)
    map2D_smooth = fftpack.fftshift(map2D_smooth) # ÑAPA

    return map2D_smooth
    
    
#-------------------------------------------------------------------        
def convert_map_microK(map2D, band):
    """Convert map to microK.""" 
    
    if (band==545) | (band==857):
        map2D_microK = map2D*(10.**6)/Conv[band]
    else:
        map2D_microK = map2D*(10.**6)
        
    return map2D_microK

    
#-------------------------------------------------------------------        
def sum_maps(redm_ids, band, smooth=True, produce_images=False, images_tag=None):
    """Smooth maps to common resolution (optional) & sum maps in band."""
    
    sum_map = np.zeros((img_pix, img_pix))

    print '\nStacking clusters in band %s...\n' % band

    for i, clu_id in enumerate(redm_ids):

        print '\r%.2f completed' % (100.*i/len(redm_ids)),
        
        map2D = cluster_patch(clu_id, band, zoom_factor)
        
        ## Smooth to common resolution:      
        if smooth:
            map2D = smooth_map(map2D, band)
                
        ## Sum to stacked map:
        sum_map += map2D
    
    print '\r100.00 completed\n'   
       
    ## Print image in img_dir:
    if produce_images:
        png_file = '%s/Planck/Stacked/stacked_map_%s_%s.png' % (img_dir, band, images_tag)
        fig = plot_image_data(sum_map, lognorm=False)
        fig.savefig(png_file)
        plt.close(fig)
        
    print '-'*20
        
    return sum_map


#-------------------------------------------------------------------
def compute_SZ_signal(maps_sum, produce_images=False):
    """Get SZ signal"""
    
    ## Define distances & masks:
    distances = distances_image(img_pix)

    R1 = 10.0 # pixels
    R2 = 14.0 # pixels
    R3 = 30.0 # pixels
    
    P1 = distances <= R1
    P2 = (distances > R1) & (distances <= R2)
    P3 = distances > R3

    NP1 = P1.sum()
    NP2 = P2.sum()
    NP3 = P3.sum()

    PR1 = np.abs(distances - R1) < 0.5
    PR2 = np.abs(distances - R2) < 0.5

    ## Define objects to store points:
    SZ_R1R2 = np.zeros(6)
    SZ_R1R2_err = np.zeros(6)

    ## Normalize maps & compute SZ values:
    for i, band in enumerate([100, 143, 217, 353, 545, 857]):

        maps_sum[band] -= np.mean(maps_sum[band][P2])

        if produce_images:
            fig = plot_image_data(maps_sum[band], lognorm=False)
            fig.savefig('%s/Planck/Stacked/maps_sum_%s_v2.png' % (img_dir, band))
            plt.close(fig)

        SZ_R1R2[i] = np.mean(maps_sum[band][P1]) - np.mean(maps_sum[band][P2]) # microK

    ## Subtract 217 & compute error:
    for i, band in enumerate([100, 143, 217, 353, 545, 857]):
        
        SZ_R1R2[i] -= SZ_R1R2[2]
        SZ_R1R2_err[i] = np.std(maps_sum[band][P3])

    SZ_R1R2_err *= 3.0/np.sqrt(NP1)
    
    return SZ_R1R2, SZ_R1R2_err

        
#-------------------------------------------------------------------
def create_SZ_patch(clu_id, img_pix, zoom_factor):
    """Creates SZ patch."""
    
    distances = distances_image(img_pix)
    PD1 = distances <= 8.5
    PD2 = distances > 8.5   
               
    map2D = {}

    for band in [100, 143, 217, 353, 545, 857]:

        map2D_0 = cluster_patch(clu_id, band, zoom_factor)

        ## Convert to microK and smooth to common resolution:      
        map2D_0 = convert_map_microK(map2D_0, band)
        map2D_smooth = smooth_map(map2D_0, band)

        map2D[band] = map2D_smooth

    ## Clean maps & create SZ map:
    dust_map2D = map2D[353] - map2D[217]
    map2D_217c = map2D[217] - 0.142*(map2D[353] - map2D[143]) # 143V3b

    C100 = 0.015
    C143 = 0.043

    SZ_response_217c = SZ_response[217] - 0.142*(SZ_response[353] - SZ_response[143])

    R100 = SZ_response[100] - SZ_response_217c - C100*SZ_response[353]
    R143 = SZ_response[143] - SZ_response_217c - C143*SZ_response[353]

    SZ_map_100 = (map2D[100] - map2D_217c - C100*dust_map2D)/R100 # Ycomton units 
    SZ_map_143 = (map2D[143] - map2D_217c - C143*dust_map2D)/R143 # Ycomton units 

    sigma_100 = 1.3*np.std(SZ_map_100)
    sigma_143 = 1.3*np.std(SZ_map_143)

    W2 = (sigma_143/sigma_100)**2 # ~ 0.?
    W3 = (sigma_143/sigma_143)**2 # ~ 1.???

    SZ_map_ILC = W2*SZ_map_100 + W3*SZ_map_143 
    SZ_map_ILC = SZ_map_ILC/(W2 + W3) # Ycompton units

    ## Smooth map:
    FFTm = fftpack.fft2(SZ_map_ILC)
    FFTm = FFTm*FFTMHW 
    SZ_map_ILC_MHW = fftpack.ifft2(FFTm)
    SZ_map_ILC_MHW = fftpack.fftshift(SZ_map_ILC_MHW) # ÑAPA

    ## Remove bright radio sources:
    sigma_SZ_map2D = np.std(SZ_map_ILC)
    mask = SZ_map_ILC < -3.*sigma_SZ_map2D
    SZ_map_ILC[mask] = np.mean(SZ_map_ILC[~mask])

    SZ_map_ILC = np.array(SZ_map_ILC, dtype=np.float16)
    
    return SZ_map_ILC

    
#-------------------------------------------------------------------        
def create_SZ_patches(redm_ids, produce_images=False):
    """If missing, creates SZ map.""" 

    print '\nCreating SZ maps...\n'

    for i, clu_id in enumerate(redm_ids):

        print '\r%.2f completed' % (100.*i/len(redm_ids)),

        ## Obtain patches:
        patch_file = '%s/Planck_patchs/%s/Patch_%06d_%02d.npy' % (data_dir, 'SZ', clu_id, zoom_factor)
        if not os.path.isfile(patch_file):
            SZ_map2D = create_SZ_patch(clu_id, img_pix, zoom_factor)
            np.save(patch_file, SZ_map2D)
                        
        ## Print image in img_dir:
        if produce_images:
            SZ_map2D = cluster_patch(clu_id, 'SZ', zoom_factor)
            SZ_png_file = '%s/Planck/PNG/%s/IMG_%06d_%02d.png' % (img_dir, 'SZ', clu_id, zoom_factor)
            fig = plot_image_data(SZ_map2D, lognorm=False)
            fig.savefig(SZ_png_file)
            plt.close(fig)

    print '\r100.00 completed\n'
    print '-'*20
    
    
#------------------------------------------------------------------- 
def compute_SZ_fluxes(SZ_R1R2, SZ_R1R2_err):
    """Compute SZ values."""

    nu_bands = np.array([100., 143., 217., 353., 545., 857.])
    X = nu_bands/56.4
    Hx = X**4*np.exp(X)/(np.exp(X) - 1.)**2
    Fx = SZ_R1R2*Hx
    Fx_err = SZ_R1R2_err*Hx

    ## Compute model values using data to normalize:
    def SZ_model():

        nu_m = np.arange(1, 1000, 1.)
        X_m = nu_m/56.4
        H_m = X_m**4*np.exp(X_m)/(np.exp(X_m) - 1.)**2
        F_NR = H_m*(X_m*(np.exp(X_m) + 1.)/(np.exp(X_m) - 1.) - 4.)
        Ya = Fx[1]/F_NR[143+1]
        Yb = Fx[3]/F_NR[353+1]
        Y = 0.7*Ya + 0.3*Yb

        SZ_m = Y*F_NR

        return nu_m, SZ_m

    nu_m, SZ_m = SZ_model()
    
    return Fx, Fx_err, SZ_m


#------------------------------------------------------------------- 
def plot_maps_sum(maps_sum, sub_meta, colormap=None):
    """Plot stacked corrected maps."""
       
    fig = plt.figure(figsize=(10, 6))
    for i, band in enumerate([100, 143, 217, 353, 545, 857]):
        ax = fig.add_subplot(2,3,i+1)
        ax.imshow(maps_sum[band], cmap=colormap)
        ax.set_title('%i GHz' % band)
        ax.get_xaxis().set_visible(False)
        ax.get_yaxis().set_visible(False)

    fig.tight_layout(h_pad=2.)
    fig.savefig('%s/Maps_sum/maps_sum_%s.png' % (plot_dir, sub_tag(sub_meta)))
    plt.close(fig)
    #save_figure(fig, 'Sum_maps_freqs')
    
#-------------------------------------------------------------------
def plot_SZ_map(SZ_map_sum, sub_meta, colormap=None):
    """Plot SZ map."""
    
    fig = plt.figure(figsize=(4, 4))
    ax = fig.add_subplot(111)
    ax.imshow(SZ_map_sum, cmap=colormap)
    ax.set_title('SZ')
    ax.get_xaxis().set_visible(False)
    ax.get_yaxis().set_visible(False)

    fig.tight_layout()
    fig.savefig('%s/SZ_maps/map_SZ_%s.png' % (plot_dir, sub_tag(sub_meta)))
    plt.close(fig)

    
#-------------------------------------------------------------------
def plot_SZ_signal(Fx, Fx_err, SZ_m, meta, colormap=None):
    """Plot SZ signal."""
    
    label = 'R: [%i, %i), z: [%.3f, %.3f), N: %i' % (meta[0], meta[1], meta[2], meta[3], meta[4])
    N = meta[4]
    
    nu_bands = np.array([100., 143., 217., 353., 545., 857.])
    nu_m = np.arange(1, 1000, 1.)
    
    fontsize = 14
    markersize = 3.

    matplotlib.rcParams['xtick.labelsize'] = fontsize
    matplotlib.rcParams['ytick.labelsize'] = fontsize

    fig = plt.figure(figsize=(8,6))
    ax = fig.add_subplot(111)

    ax.set_title(label, fontsize=fontsize)
    ax.set_xlim([0., 1000.])
    ax.set_xlabel(r'$\nu$ [GHz]', fontsize=fontsize)
    ax.set_ylim([2.*(SZ_m/N).min(), 8.*(SZ_m/N).max()])
    ax.set_ylabel(r'SZ / N [microK]', fontsize=fontsize)

    ax.plot(nu_m, np.zeros(len(nu_m)), 'k-', alpha=0.3)
    ax.errorbar(nu_bands, Fx/N, yerr=Fx_err/N, fmt='k--', marker='o', markersize=markersize, label='data')
    ax.plot(nu_m, SZ_m/N, 'r:', label='model')

    plt.legend(loc=2, fontsize=fontsize, numpoints=1)

    fig.tight_layout()
    fig.savefig('%s/SZ_signal/SZ_signal_%s.png' % (plot_dir, sub_tag(meta)))
    plt.close(fig)
    
    
#-------------------------------------------------------------------
def plot_results(maps_sum, SZ_map_sum, Fx, Fx_err, SZ_m, sub_meta, colormap=None):
    """Plot global results."""
    
    fontsize = 14
    markersize = 3.

    matplotlib.rcParams['xtick.labelsize'] = fontsize
    matplotlib.rcParams['ytick.labelsize'] = fontsize
    
    fig = plt.figure(figsize=(12, 12))
    #sub_meta : [rich_min, rich_max, z_min, z_max, N_clu, rich_mean, z_mean]
    fig_title = 'R: [%.1f, %.1f),   <R> = %.1f,   z: [%.3f, %.3f),   <z> = %.3f,   N: %i' % (
    sub_meta[0], sub_meta[1], sub_meta[5], sub_meta[2], sub_meta[3], sub_meta[6], sub_meta[4])
    fig.suptitle(fig_title, fontsize=fontsize+2, fontweight='bold')
    
    ## Plot band maps:
    for i, band in enumerate([100, 143, 217, 353, 545, 857]):
        ax = fig.add_axes([0.03+0.22*(i%3), 0.69-0.23*(i/3), 0.2, 0.25])
        crop_image = maps_sum[band][10:-10,10:-10]
        ax.imshow(crop_image, cmap=colormap)
        ax.set_title('%i GHz' % band)
        ax.get_xaxis().set_visible(False)
        ax.get_yaxis().set_visible(False)
        
    ## Plot SZ map:
    ax = fig.add_axes([0.70, 0.55, 0.25, 0.35])
    crop_image = SZ_map_sum[10:-10,10:-10]
    ax.imshow(crop_image, cmap=colormap)
    ax.set_title('SZ')
    ax.get_xaxis().set_visible(False)
    ax.get_yaxis().set_visible(False)
    
    ## Plot SZ signal:    
    N = sub_meta[4]
    
    nu_bands = np.array([100., 143., 217., 353., 545., 857.])
    nu_m = np.arange(1, 1000, 1.)
    
    ax = fig.add_axes([0.13, 0.06, 0.8, 0.40])
    ax.set_xlim([0., 1000.])
    ax.set_xlabel(r'$\nu$ [GHz]', fontsize=fontsize)
    ax.set_ylim([2.*(SZ_m/N).min(), 8.*(SZ_m/N).max()])
    ax.set_ylabel(r'SZ / N [microK]', fontsize=fontsize)

    ax.plot(nu_m, np.zeros(len(nu_m)), 'k-', alpha=0.3)
    ax.errorbar(nu_bands, Fx/N, yerr=Fx_err/N, fmt='k--', marker='o', markersize=markersize, label='data')
    ax.plot(nu_m, SZ_m/N, 'r:', label='model')

    plt.legend(loc=2, fontsize=fontsize, numpoints=1)

    plot_file = '%s/Samples_results/Sample_%s.png' % (plot_dir, sub_tag(sub_meta))
    fig.savefig(plot_file)
    plt.close(fig)
    
    print '\nResults figure saved in %s\n' % plot_file
    
    
#-------------------------------------------------------------------
def run_code(subsample_bins, REDO_stacking):
    """RUN CODE"""    
    
    sample_metas = []
    SZ_R1R2_vals = []
    SZ_R1R2_err_vals = []
    
    for sub_bin in subsample_bins:
          
        rich_min, rich_max, z_min, z_max = sub_bin
        
        if (rich_min>=rich_max) or (z_min>=z_max):
            continue
    
        #-------------------------------------------------------------------
        ## Create subsample:
        sub_ids, sub_meta = define_redm_subsample(rich_min, rich_max, z_min, z_max)
        #sub_meta : [rich_min, rich_max, z_min, z_max, N_clu, rich_mean, z_mean]

        #-------------------------------------------------------------------
        ## Obtain missing patches from Planck maps:
        if 0:
            for band in planck_maps.keys():
                create_patches(sub_ids, band, produce_images=False)

        ## Stack maps:
        maps_sum_file = '%s/Planck_stacked_maps/maps_sum_%s.pickle' % (data_dir, sub_tag(sub_meta))

        if not os.path.isfile(maps_sum_file) or (REDO_stacking):  
            maps_sum = {}
            for band in [100, 143, 217, 353, 545, 857]:
                maps_sum[band] = sum_maps(sub_ids, band, produce_images=True, images_tag=sub_tag(sub_meta))
            with open(maps_sum_file, 'wb') as handle:
                pickle.dump(maps_sum, handle, protocol=pickle.HIGHEST_PROTOCOL)
        else:
            with open(maps_sum_file, 'rb') as handle:
                maps_sum = pickle.load(handle)

        ## Convert stacked maps to microK:        
        for band in [100, 143, 217, 353, 545, 857]:
            maps_sum[band] = convert_map_microK(maps_sum[band], band)

        ## Clean band 217 from dust:
        maps_sum[217] -= maps_sum[857]*(10.**-4)

        #-------------------------------------------------------------------
        ## Compute SZ signal:
        SZ_R1R2, SZ_R1R2_err = compute_SZ_signal(maps_sum, produce_images=True)

        ## Stack SZ maps:
        SZ_map_sum_file = '%s/Planck_stacked_maps/SZ_map_sum_%s.pickle' % (data_dir, sub_tag(sub_meta))

        if not os.path.isfile(SZ_map_sum_file) or (REDO_stacking):  
            create_SZ_patches(sub_ids, produce_images=False)
            SZ_map_sum = sum_maps(sub_ids, 'SZ', smooth=False, produce_images=True, images_tag=sub_tag(sub_meta))
            with open(SZ_map_sum_file, 'wb') as handle:
                pickle.dump(SZ_map_sum, handle, protocol=pickle.HIGHEST_PROTOCOL)
        else:
            with open(SZ_map_sum_file, 'rb') as handle:
                SZ_map_sum = pickle.load(handle)
                
        #-------------------------------------------------------------------
        ## Export results:
        store_SZ_data = [sub_meta, SZ_R1R2, SZ_R1R2_err]
        store_SZ_file = '%s/SZ_values/SZ_data_%s.pickle' % (data_dir, sub_tag(sub_meta))
        with open(store_SZ_file, 'wb') as handle:
            pickle.dump(store_SZ_data, handle, protocol=pickle.HIGHEST_PROTOCOL)

        #-------------------------------------------------------------------  
        ## Plot results:
        Fx, Fx_err, SZ_m = compute_SZ_fluxes(SZ_R1R2, SZ_R1R2_err)

        plot_maps_sum(maps_sum, sub_meta, colormap=None)
        plot_SZ_map(SZ_map_sum, sub_meta, colormap=None)
        plot_SZ_signal(Fx, Fx_err, SZ_m, sub_meta, colormap=None)

        plot_results(maps_sum, SZ_map_sum, Fx, Fx_err, SZ_m, sub_meta, colormap=None)

        sample_metas.append(sub_meta)
        SZ_R1R2_vals.append(SZ_R1R2)
        SZ_R1R2_err_vals.append(SZ_R1R2_err)
    
        print '*'*50
        
    return sample_metas, SZ_R1R2_vals, SZ_R1R2_err_vals
        
    
#-------------------------------------------------------------------
def plot_SZ_comparison(sample_metas, SZ_R1R2_vals, SZ_R1R2_err_vals, nr_bins=3, nz_bins=2):
    
    fontsize = 12
    markersize = 3.

    matplotlib.rcParams['xtick.labelsize'] = fontsize
    matplotlib.rcParams['ytick.labelsize'] = fontsize
    
    nu_bands = np.array([100., 143., 217., 353., 545., 857.])
    nu_m = np.arange(1, 1000, 1.)
        
    fig = plt.figure(figsize=(nz_bins*4, nr_bins*3))
    
    for i, meta in enumerate(sample_metas):

        Fx, Fx_err, SZ_m = compute_SZ_fluxes(SZ_R1R2_vals[i], SZ_R1R2_err_vals[i])
        N = meta[4]
        label = 'R: [%i, %i), z: [%.3f, %.3f), N: %i' % (meta[0], meta[1], meta[2], meta[3], meta[4])
        
        ## Order subsamples:
        ax = fig.add_subplot(nr_bins, nz_bins, nr_bins*nz_bins - (i/nz_bins+1)*nz_bins + i%nz_bins + 1)
        ax.set_title(label, fontsize=fontsize)
        ax.set_xlim([0., 1000.])
        ax.set_xlabel(r'$\nu$ [GHz]', fontsize=fontsize)
        ax.set_ylim([2.*(SZ_m/N).min(), 8.*(SZ_m/N).max()])
        ax.set_ylabel(r'SZ / N [microK]', fontsize=fontsize)
        ax.plot(nu_m, np.zeros(len(nu_m)), 'k-', alpha=0.3)
        
        ax.errorbar(nu_bands, Fx/N, yerr=Fx_err/N, fmt='k:', markersize=markersize)#label=label)
        ax.plot(nu_m, SZ_m/N, 'r:')
        #lt.legend(loc=2, fontsize=fontsize, numpoints=1)

    plot_file = '%s/SZ_comparison/SZ_comparison_[nr:_%i_nz:_%i].png' % (plot_dir, nr_bins, nz_bins)
    fig.tight_layout()
    fig.savefig(plot_file)
    plt.close()
    
    
#-------------------------------------------------------------------

REDO_stacking = False

#R_vals = [20, 30, 50, 300]
#R_vals = [20, 30, 50, 100, 300]
#R_vals = [20, 30, 45, 70, 100, 150, 300]

#z_vals = [0.100, 0.350, 0.550]
#z_vals = [0.100, 0.250, 0.400, 0.550]
#z_vals = [0.100, 0.200, 0.300, 0.400, 0.500, 0.600]

#subsample_bins, nr_bins, nz_bins = automatic_binning(R_vals, z_vals) 

## Run code:
sample_metas, SZ_R1R2_vals, SZ_R1R2_err_vals = run_code(subsample_bins, REDO_stacking)
plot_SZ_comparison(sample_metas, SZ_R1R2_vals, SZ_R1R2_err_vals, nr_bins, nz_bins)